# LINEAR REGRESSION
<B>Objective:</B> To gets hands on experience on linear regression. 
* Cost Function for regression
* Gradient Descent, Learning rates
* Feature Normalization
* Iterative method vs Direct Method for regression
* Using scikit-learn library for regression
* How to work with categorical variables / one hot encoding

<B>Problem:</B> We will solve two set of problems 
1. Predicting house price
2. Predicting weather station maintenence request counts

# INTRODUCTION
Linear Regression is a supervised machine learning algorithm where the predicted output is continuous and has a constant slope. Is used to predict values within a continuous range. (e.g. sales, price) 

<b>Simple Regression:</b> Simple linear regression uses traditional slope-intercept form, where $m$ and $b$ are the variables our algorithm will try to “learn” to produce the most accurate predictions. $x$ represents our input data and $y$ represents our prediction.

$$\hat {y}=\theta_{1}x+\theta_{0}$$

In order to compute the values of $m$ and $b$ we need to minimize a cost function. In the case of Linear Regression it is given by

$$J(\theta) = \frac{1}{2m}\Sigma_{i=i}^{m}(\hat {y}_{i}-y_{i})^{2}$$

<img src='./images/lr.jpg'>

In multivariate regression we seek a set of parameters 

$\textbf{$\Theta$} = [\theta_{0} \quad \theta_{1} \quad \cdots \quad \theta_{i} \quad \cdots \quad \theta_{m}]^{T}$
               
which minimizes the the cost function:
Hypothesis 
$\textbf{H}=\textbf{X $\Theta$}$

Cost function is $\textbf{J($\Theta$)}=\frac{1}{2m}(\textbf{X$\Theta$} -\textbf{Y})^{T}(\textbf{X$\Theta$} -\textbf{Y})$

The advantages of Linear Regression are that they computationally efficient, simple and easy to interpret. However, the algorithm fails to capture non-linear behavior.


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
#from sklearn import datasets,linear_model

# Loading the data from a csv file
The columns represent the surface area, number of rooms and the price of the apartment. 

In [ ]:
data = np.loadtxt('./data/housing_data.csv', delimiter=",")
X = data[:,:2]
y = data[:,2]
m = len(y)

# Data Exploration
Plot the data and have a look
Note the large difference in the scale of "number of room variables" and "surface area". 
* "Number of room" has a range [1.0,5.0]
* "Surface area" has a range [500,4500]

This causes problems during the cost minimization step (gradient descent).

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(121)
plt.scatter(X[:,1],y)
plt.xlabel("Number of rooms")
plt.ylabel("Price of the apartment")
plt.title("Price of the apartment vs number of rooms")
plt.subplot(122)
plt.scatter(X[:,0],y)
plt.xlabel("Surface area")
plt.ylabel("Price of the apartment")
plt.title("Price of the apartment vs surface area")
plt.show()

# Feature normalization. 
The input feature vectors are normalized so that they have comparable scales.Here we do so by subtracting the mean and dividing by the standard deviation.

In [ ]:
# Normalize features
def featureNormalize(X):
    X_norm = X.copy()
    mu    = np.zeros((1, X.shape[1]))
    sigma = np.zeros((1, X.shape[1]))
    for i in range(X.shape[1]):
        mu[:,i] = np.mean(X[:,i])
        sigma[:,i] = np.std(X[:,i])
        X_norm[:,i] = (X[:,i] - float(mu[:,i]))/float(sigma[:,i])
    return X_norm, mu, sigma

In [ ]:
X_norm, mu, sigma = featureNormalize(X)

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(121)
plt.scatter(X_norm[:,1],y)
plt.xlabel("Normalized number of rooms")
plt.ylabel("Price of the apartment")
plt.title("Price of the apartment vs normalized number of rooms")
plt.subplot(122)
plt.scatter(X_norm[:,0],y)
plt.xlabel("Normalized surface area")
plt.ylabel("Price of the apartment")
plt.title("Price of the apartment vs Normalized surface area")
plt.show()

# Bias Term
We simply add a column vector of 1.

In [ ]:
X_padded = np.column_stack((np.ones((m,1)), X_norm))

# Cost Function involving multiple variables
$\textbf{J($\Theta$)}=\frac{1}{2m}(\textbf{X$\Theta$} -\textbf{Y})^{T}(\textbf{X$\Theta$} -\textbf{Y})$

In [ ]:
def computeCost(X,y,theta): 
  m=len(y)
  Cost=0.0;
  Cost=1.0/2.0/m*np.sum((X.dot(theta)-y)**2);
  return Cost;

# Batch Gradient Descent
Minimize the loss function using gradient descent. 
<img src='images/gd.jpg'>

In [ ]:
def gradientDescent(X, y, alpha, num_iters):
    m,n = X.shape
    theta = np.zeros((X.shape[1],1))
    Cost_history = np.zeros((num_iters, 1))
    for i in range(num_iters):
        theta = theta - alpha*(1.0/m) * np.transpose(X).dot(X.dot(theta) - np.transpose([y]))    
        Cost_history[i] = computeCost(X, y, theta)
    return theta, Cost_history

In [ ]:
def SolveGradientDescent(X,y,alpha, num_iters):
    m,n=X.shape
    theta, Loss_history = gradientDescent(X, y, alpha, num_iters)
    plt.plot(range(Loss_history.size), Loss_history, "-b", linewidth=2 )
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost')
    plt.show(block=False)
    theta.shape
    return theta

# Evolution of the cost function
Try solving the same problem but without feature normalization and convince yourself the importance of the step.

In [ ]:
theta_GD=SolveGradientDescent(X=X_padded,y=y,alpha=0.1,num_iters=70)
print('Theta computed from gradient descent: ',theta_GD.T)

# Make predictions using the trained model
Remember that the input features were normalized so we need to do the same with the new input features on which we want to make make predictions.

In [ ]:
house_norm_padded = np.array([1, (1650-mu[0,0])/sigma[0,0], (3-mu[0,1])/sigma[0,1]])
price_GD = np.array(house_norm_padded).dot(theta_GD)
print("Predicted price of a 1650 sq-ft, 3 br house (using gradient descent):", price_GD)

# Interpretability of the model
The weights computed above corresponds to the normalized features and is given by.
$\hat{y}=\theta_{0}+\theta_{1}(\frac{x_{1}-\mu_{1}}{\sigma_{1}})+\theta_{2}(\frac{x_{2}-\mu_{2}}{\sigma_{2}})$

To interpret the model we should look at the weights corresponding to the actual features.
$\hat{y}=\theta_{0}+\theta_{1}x_{1}+\theta_{2}x_{2}$, then the coefficients will be as follows

In [ ]:
theta_GD[0]-mu[0,0]/sigma[0,0]-mu[0,1]/sigma[0,1],theta_GD[1]/sigma[0,0],theta_GD[1]/sigma[0,1]

What is your conclusion from the comparison of the weights ?

# Solution using Normal Equation
Generally for very large problems due to memory requirements gradient descent algorithm is used for minimization but since here we are working with relatively small dataset with smaller number of feature vectors $X$, we have used direct method for finiding the minima. This involves invoking the closed-form solution to linear regression. Convince yourself with a little calculation that:

$\Theta=(X^{T}X)^{-1}X^{T}Y$

In [ ]:
# Solve using direct method
def normalEqn(X, y):  
    theta = np.zeros((X.shape[1], 1))
    theta = np.linalg.pinv(np.transpose(X).dot(X)).dot(np.transpose(X).dot(y))
    return theta

In [ ]:
theta_Normal = normalEqn(X_padded, y)
print("Theta calculated by Normal Equation ",theta_Normal)
price_Normal = np.array(house_norm_padded).dot(theta_Normal)
print("Predicted price of a 1650 sq-ft, 3 br house (using normal equation):", price_Normal)

# Regression using scikit-learn library in two lines

In [ ]:
from sklearn import linear_model
lr = linear_model.LinearRegression()
lr.fit(X_padded,y)
print("Theta calculated by SK-learn regression ",lr.coef_)
print("Predicted price ",lr.predict(house_norm_padded.reshape(1,-1)))

# Implement Stochastic Gradient Descent
<img src='./images/stochasticGD.jpg'>

# Example 2


Data Description
    
* date : yyyy-mm-dd format
* calendar_code : 0 or 1 (a code describing certain calendar events)
* request_count : an integer (the number of support requests received on that date)
* site_count : an integer (the number of sites operating on that date)
* max_temp : a float (max temperature for that day in degrees Celsius)
* min_temp : a float (min temperature for that day in degrees Celsius)
* precipitation : a float (millimeters of precipitation on that date)
* events : a string (description of weather events on that date)

Our aim is to predict the request_count when the other paeameters are given.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#ALso remember to parse the date column. This will be helpful in the next step
training_data=pd.read_csv('./data/Met_train.csv',sep=',',parse_dates=['date'])
training_data.head()

# Data Exploration

In [ ]:
training_data.hist('max_temp',weights=training_data['request_count'],figsize=(16,4))
training_data.hist('min_temp',weights=training_data['request_count'],figsize=(16,4))
training_data.hist('precipitation',weights=training_data['request_count'],figsize=(16,4))
plt.show()


From the above histograms we see that most of the request comes when 
1. maximum temperature is below 10C 
2. min temperature is below 2C 
3. When there is zero precipitation

We use violin plot for dependence on the categorical variables (https://blog.modeanalytics.com/violin-plot-examples/)

In [ ]:
var_name = "events"
col_order = np.sort(training_data[var_name].unique()).tolist()
plt.figure(figsize=(16,6))
sns.violinplot(x=var_name, y='request_count', data=training_data, order=col_order)
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of request count with "+var_name, fontsize=15)
plt.show()

In [ ]:
var_name = "calendar_code"
col_order = np.sort(training_data[var_name].unique()).tolist()
plt.figure(figsize=(16,6))
sns.violinplot(x=var_name, y='request_count', data=training_data, order=col_order)
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of request count with "+var_name, fontsize=15)
plt.show()

In [ ]:
training_data['day_of_week'] = training_data['date'].dt.dayofweek
training_data['week_day'] = training_data['date'].dt.weekday_name
training_data.head()

In [ ]:
var_name = "week_day"
col_order = np.sort(training_data[var_name].unique()).tolist()
plt.figure(figsize=(16,6))
sns.violinplot(x=var_name, y='request_count', data=training_data, order=col_order)
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of request count with "+var_name, fontsize=15)
plt.show()

# Dealing with categorical variables
In most of the machine learning algorithms barringa a few (ex. Decision Tree) we need numerical values for the input features. In the current data events are categorical variable which should be converted into a unique numerical identifiers. This will result in an additional column "events_code"

In [ ]:
training_data['events_code'] = pd.Categorical(training_data["events"]).codes
training_data.head()

Since request count is the target variable, we store it separately as "y"

In [ ]:
y=training_data["request_count"]

Drop the redundant columns now "date","events","request_count"

In [ ]:
training_data = training_data.drop(["date","events","request_count","week_day"],axis=1)
training_data.head()

# One-Hot-Encoding
The numerical values of day_of_week, events_code and calender code do not signify anything so they need to be one-hot-encoded to be used as a feature input vector.

In [ ]:

training_data= pd.get_dummies(training_data,columns=["calendar_code","events_code","day_of_week"],prefix=["calendar","event","week"])
training_data.head()

Call the feature vectors X

In [ ]:
X=training_data.values
plt.figure(figsize=(16,8))
plt.subplot(131)
plt.plot(X[:,0],y[:],'r.')
plt.xlabel("No. of sites")
plt.ylabel("No. of requests")
plt.subplot(132)
plt.plot((X[:,1]+X[:,2])/2.0,y[:],'r.')
plt.xlabel("Mean temperature")
plt.ylabel("No. of requests")
plt.subplot(133)
plt.plot(X[:,3],y[:],'r.')
plt.xlabel("Precipitation")
plt.ylabel("No. of requests")
plt.show()

# Feature Engineering
It appeats that the no of requests has some kind of a quadratic dependence on the mean temperature so in addition to max and min temperature we should construct a new feature $((minx+maxx)/2)^{2}$. Feature engineering is one way of brining in domain knowledge, experience or gut feeling into the analysis.

In [ ]:
X=np.column_stack([X,(X[:,1]+X[:,2])**2.0])

# Splitting the data into training and test set
Generally we can always tune the hyperparameters so much that the model performs well on the data but it fails to generalize on the unseen data. To avoid this we split the available data into traing and test sets. We want somewhat similar performance on both the sets.

In [ ]:
#Split the data into training and validation test
X_train, X_test, y_train, y_test =  train_test_split(X,y,test_size=0.2,random_state = 0)

Conduct a multivariate linear regression on the dataset.

In [ ]:
#Multivariabte regression
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
y_train_pred=regr.predict(X_train)
print("Mean squared error on training set: %.2f" % np.mean((regr.predict(X_train) - y_train) ** 2))
print("Mean squared error on validation set: %.2f" % np.mean((regr.predict(X_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score on training set: %.2f' % regr.score(X_train, y_train))
print('Variance score on validation set: %.2f' % regr.score(X_test, y_test))

In [ ]:
pred_LR = regr.predict(X_test)
y_val = y_test.values.reshape(-1,1)

In [ ]:
for i in range(len(pred_LR)):
    print(y_val[i],pred_LR[i])